In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from radio_data_utility import RadioV1Dataset as dataset
import argparse
import os
import json
import pickle

from torchvision import transforms

import torch
import numpy as np

from torch.utils.data import DataLoader
import torch.nn as nn

from tqdm import tqdm

import time
import random


In [3]:
with open('./generated_radio_datasets_split/raven_organsmnist_v1_train.pkl', 'rb') as f:
    raven_dataset_v1_organ_train = pickle.load(f)

with open('./generated_radio_datasets_split/raven_organsmnist_v1_val.pkl', 'rb') as f:
    raven_dataset_v1_organ_val = pickle.load(f)

with open('./generated_radio_datasets_split/raven_organsmnist_v1_test.pkl', 'rb') as f:
    raven_dataset_v1_organ_test = pickle.load(f)

In [4]:
from mxgnet import Model, train_epoch, validate, count_parameters
from radam import RAdam

In [5]:
RANDOM_SEED = 42

# Set seeds for reproducibility
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

In [6]:
IMG_SIZE = 80

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_SIZE),  # Resize to the specified size
    transforms.ToTensor(),
])

eval_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])

In [7]:
print("Total samples in training set:", len(raven_dataset_v1_organ_train))
print("Total samples in validation set:", len(raven_dataset_v1_organ_val))
print("Total samples in test set:", len(raven_dataset_v1_organ_test))

Total samples in training set: 4400
Total samples in validation set: 550
Total samples in test set: 550


In [8]:
train_dataset = dataset(
    raven_dataset_v1_organ_train, # Use the split training data
    mode="train",
    transform_train=train_transform,
    transform_eval=eval_transform, # transform_eval is not used by train mode but good to pass
)


val_dataset = dataset(
    raven_dataset_v1_organ_val, # Use the split validation data
    mode="val",    # Set mode to "val" or "test"
    transform_train=train_transform, # Not used by val mode
    transform_eval=eval_transform,
)

test_dataset = dataset(
    raven_dataset_v1_organ_test, # Use the split test data
    mode="val",    # Set mode to "val" or "test"
    transform_train=train_transform, # Not used by test mode
    transform_eval=eval_transform,
)

In [9]:
import torch
import torch.nn as nn
import numpy as np
import time
import json
from sklearn.model_selection import StratifiedKFold, train_test_split
# ==============================================================================
# 1. SETUP - Constants and Parameters
# ==============================================================================
N_SPLITS = 3 # Number of folds for cross-validation
PATIENCE = 10 # Early stopping patience
VAL_SPLIT_SIZE = 0.20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 100
LEARNING_RATE = 1e-4
BATCH_SIZE = 128

In [10]:
from torch.utils.data import TensorDataset, ConcatDataset, Subset, DataLoader

dev_dataset = ConcatDataset([train_dataset, val_dataset])
dev_dataset_labels = np.concatenate([train_dataset.labels, val_dataset.labels])

print(f"Total data for Development (Train+Val): {len(dev_dataset)} samples.")
print(f"Test Set size: {len(test_dataset)} samples. (Will not be used until the very end)")

Total data for Development (Train+Val): 4950 samples.
Test Set size: 550 samples. (Will not be used until the very end)


# RADIO-1 STL

In [11]:
model_folder_name = "RADIO1_MXGNet"
from tensorboardX import SummaryWriter
import time

In [13]:
from tensorboardX import SummaryWriter
import time


print(f"\n{'='*25} PERFORMING NESTED CROSS-VALIDATION {'='*25}")

outer_kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
fold_test_results = []

for fold, (train_outer_idx, test_outer_idx) in enumerate(outer_kfold.split(np.zeros(len(dev_dataset)), dev_dataset_labels)):
    print(f"\n--- Outer Fold {fold+1}/{N_SPLITS} ---")
    writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/fold_{fold+1}')

    # Create inner train/val split from the outer training set
    train_outer_labels = dev_dataset_labels[train_outer_idx]
    train_inner_idx, val_inner_idx = train_test_split(
        train_outer_idx, test_size=VAL_SPLIT_SIZE, shuffle=True, stratify=train_outer_labels, random_state=RANDOM_SEED
    )

    # Create subsets from the DEVELOPMENT dataset
    train_inner_subset = Subset(dev_dataset, train_inner_idx)
    val_inner_subset = Subset(dev_dataset, val_inner_idx)
    test_outer_subset = Subset(dev_dataset, test_outer_idx) # This is the "test set" for this fold

    train_loader = DataLoader(train_inner_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
    val_loader = DataLoader(val_inner_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)
    test_loader = DataLoader(test_outer_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)

    model = Model(IMG_SIZE, IMG_SIZE,beta=0, batch_size=BATCH_SIZE).to(DEVICE)
    optimizer = RAdam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.NLLLoss()
    scheduler = None
    scaler = None


    best_model_path = f"./saved_models/{model_folder_name}/best_model_fold_{fold+1}.pth"
    os.makedirs(f"./saved_models/{model_folder_name}", exist_ok=True)
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    print(f"Training on {len(train_inner_subset)}, Validating on {len(val_inner_subset)}...")
    for epoch in range(1, EPOCHS + 1):
        train_start_time = time.time()
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        train_end_time = time.time()
        train_time = train_end_time - train_start_time

        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        writer.add_scalar('CV/Loss/train', train_loss, epoch)
        writer.add_scalar('CV/Accuracy/train', train_acc, epoch)
        writer.add_scalar('CV/Loss/val', val_loss, epoch)
        writer.add_scalar('CV/Accuracy/val', val_acc, epoch)
        writer.add_scalar('CV/Time/train', train_time, epoch)

        memory_allocated = torch.cuda.memory_allocated(DEVICE) if torch.cuda.is_available() else 0
        memory_reserved = torch.cuda.memory_reserved(DEVICE) if torch.cuda.is_available() else 0

        writer.add_scalar('CV/Memory/allocated', memory_allocated, epoch)
        writer.add_scalar('CV/Memory/reserved', memory_reserved, epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  New best model for fold {fold+1} at epoch {epoch}: Val Acc: {val_acc:.4f}")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    # Evaluate the best model for this fold on the outer test set
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    model.load_state_dict(torch.load(best_model_path))
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance on Outer Test Set for fold {fold+1}: {final_fold_test_acc:.4f}")
    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('CV/Accuracy/test', final_fold_test_acc, epoch)
    writer.close()

mean_cv_acc = np.mean(fold_test_results)
std_cv_acc = np.std(fold_test_results)
print(f"\n--- NESTED CROSS VALIDATION COMPLETE ---")
print(f"Cross-validation accuracies on outer folds: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Mean CV Accuracy: {mean_cv_acc:.4f} ± {std_cv_acc:.4f}")


========================= PERFORMING NESTED CROSS-VALIDATION =========================

--- Outer Fold 1/3 ---
Training on 2640, Validating on 660...


Training:   0%|          | 0/21 [00:00<?, ?it/s]/vol/bitbucket/icc24/MSRGNN/RADIO/mxgnet.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(h_score)
/vol/bitbucket/icc24/MSRGNN/RADIO/radam.py:47: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value = 1) (Triggered internally at /pytorch/torch/csrc/utils/python_arg_parser.cpp:1661.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
Training: 100%|██████████| 21/21 [00:08<00:00,  2.61it/s]


Epoch Loss: 2.2465, Accuracy: 0.1197


Validating: 100%|██████████| 6/6 [00:01<00:00,  5.66it/s]


  New best model for fold 1 at epoch 1: Val Acc: 0.1212


...

Epoch Loss: 1.6292, Accuracy: 0.3549


Validating: 100%|██████████| 6/6 [00:01<00:00,  5.64it/s]


  Early stopping at epoch 61.
Loading best model for fold 3 (achieved 0.2924 on inner val set).


Validating: 100%|██████████| 13/13 [00:01<00:00,  7.25it/s]

Performance on Outer Test Set for fold 3: 0.2339

--- NESTED CROSS VALIDATION COMPLETE ---
Cross-validation accuracies on outer folds: ['0.1176', '0.2612', '0.2339']
Mean CV Accuracy: 0.2042 ± 0.0623


In [ ]:
print(f"\n{'='*25} TRAINING FINAL DEPLOYABLE MODEL {'='*25}")

# Create the one-time 90/10 split from the development pool
final_train_indices, final_val_indices = train_test_split(
    np.arange(len(dev_dataset)), test_size=0.1, shuffle=True, stratify=dev_dataset_labels, random_state=RANDOM_SEED
)
final_train_subset = Subset(dev_dataset, final_train_indices)
final_val_subset = Subset(dev_dataset, final_val_indices)
final_train_loader = DataLoader(final_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
final_val_loader = DataLoader(final_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)

final_model = Model(IMG_SIZE, IMG_SIZE,beta=0, batch_size=BATCH_SIZE).to(DEVICE)
optimizer = RAdam(final_model.parameters(), lr=LEARNING_RATE)
criterion = nn.NLLLoss()
scheduler = None
scaler = None

writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/final_model_training')

best_model_path = f"./saved_models/{model_folder_name}/deployable_model.pth"
best_final_val_acc = 0.0
epochs_without_improvement = 0

print(f"Final training on {len(final_train_subset)} samples, validating on {len(final_val_subset)} for early stopping.")
for epoch in range(1, EPOCHS + 1):
    train_start_time = time.time()
    train_loss, train_acc = train_epoch(final_model, final_train_loader, optimizer, criterion, DEVICE)
    train_end_time = time.time()
    train_time = train_end_time - train_start_time
    
    val_loss, val_acc = validate(final_model, final_val_loader, criterion, DEVICE)
    print(f"Epoch {epoch}: Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    
    writer.add_scalar('FinalTrain/Loss/train', train_loss, epoch)
    writer.add_scalar('FinalTrain/Accuracy/train', train_acc, epoch)
    writer.add_scalar('FinalTrain/Loss/val', val_loss, epoch)
    writer.add_scalar('FinalTrain/Accuracy/val', val_acc, epoch)

    writer.add_scalar('FinalTrain/Time/train', train_time, epoch)

    memory_allocated = torch.cuda.memory_allocated(DEVICE) if torch.cuda.is_available() else 0
    memory_reserved = torch.cuda.memory_reserved(DEVICE) if torch.cuda.is_available() else 0

    writer.add_scalar('FinalTrain/Memory/allocated', memory_allocated, epoch)
    writer.add_scalar('FinalTrain/Memory/reserved', memory_reserved, epoch)

    if val_acc > best_final_val_acc:
        best_final_val_acc = val_acc
        epochs_without_improvement = 0
        torch.save(final_model.state_dict(), best_model_path)
        print(f"  New best model saved with val acc: {best_final_val_acc:.4f}")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= PATIENCE:
            print(f"  Early stopping final training at epoch {epoch}.")
            break
writer.close()
print(f"\n--- FINAL DEPLOYABLE MODEL TRAINING COMPLETE ---")
print(f"Final deployable model saved to '{best_model_path}'")


========================= TRAINING FINAL DEPLOYABLE MODEL =========================
Final training on 4455 samples, validating on 495 for early stopping.


Training:   0%|          | 0/35 [00:00<?, ?it/s]/vol/bitbucket/icc24/FINAL_RADIO_4/mxgnet.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(h_score)
Training: 100%|██████████| 35/35 [00:24<00:00,  1.42it/s]


Epoch Loss: 2.2185, Accuracy: 0.1237


Validating: 100%|██████████| 4/4 [00:00<00:00,  4.58it/s]


Epoch 1: Train Acc: 0.1237 | Val Acc: 0.1354
  New best model saved with val acc: 0.1354


...

Epoch 42: Train Acc: 0.3089 | Val Acc: 0.2687


Training: 100%|██████████| 35/35 [00:24<00:00,  1.43it/s]


Epoch Loss: 1.6945, Accuracy: 0.3178


Validating: 100%|██████████| 4/4 [00:00<00:00,  4.51it/s]

Epoch 43: Train Acc: 0.3178 | Val Acc: 0.2485
  Early stopping final training at epoch 43.

--- FINAL DEPLOYABLE MODEL TRAINING COMPLETE ---
Final deployable model saved to 'RADIO1_MXGNet/deployable_model.pth'


In [ ]:
print(f"\n{'='*25} FINAL TEST SET EVALUATION {'='*25}")

best_model_path = f"./saved_models/{model_folder_name}/deployable_model.pth"
criterion = nn.NLLLoss()

# Load the best model we just created
deployable_model = Model(IMG_SIZE, IMG_SIZE, beta=0, batch_size=BATCH_SIZE).to(DEVICE)
deployable_model.load_state_dict(torch.load(best_model_path))

# Create a dataloader for the untouched test set
test_start_time = time.time()
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)
test_end_time = time.time()
test_time = test_end_time - test_start_time

# Evaluate ONCE
_, final_benchmark_score = validate(deployable_model, test_loader, criterion, DEVICE)

writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/final_benchmark')
# writer.add_hparams(
#     {'model': 'Final Deployable', 'evaluation': 'Official Test Set'},
#     {'hparam/final_accuracy': final_benchmark_score},
#     {'hparam/final_test_time': test_time}
# )

writer.add_scalar('FinalBenchmark/Accuracy/test', final_benchmark_score, 0)
writer.add_scalar('FinalBenchmark/Time/test', test_time, 0)


writer.close()

print(f"\n--- FINAL TEST SET EVALUATION COMPLETE ---")
print(f"The final accuracy of the deployable model on the test set is: {final_benchmark_score:.4f}")


========================= FINAL TEST SET EVALUATION =========================


Validating:   0%|          | 0/5 [00:00<?, ?it/s]/vol/bitbucket/icc24/FINAL_RADIO_4/mxgnet.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(h_score)
Validating: 100%|██████████| 5/5 [00:01<00:00,  4.03it/s]


--- FINAL TEST SET EVALUATION COMPLETE ---
The final accuracy of the deployable model on the test set is: 0.2491


: 

# I-RAVEN to RADIO-1 TL

In [ ]:
from tensorboardX import SummaryWriter
import time

transfer_model_path = "../I_RAVEN/saved_models/best_model_mxgnet.pth"
model_folder_name = "RADIO1_MXGNet_TRANSFER_RAVEN"

print(f"\n{'='*25} PERFORMING NESTED CROSS-VALIDATION TRANSFER {'='*25}")

outer_kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
fold_test_results = []

for fold, (train_outer_idx, test_outer_idx) in enumerate(outer_kfold.split(np.zeros(len(dev_dataset)), dev_dataset_labels)):
    print(f"\n--- Outer Fold {fold+1}/{N_SPLITS} ---")
    writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/fold_{fold+1}')

    # Create inner train/val split from the outer training set
    train_outer_labels = dev_dataset_labels[train_outer_idx]
    train_inner_idx, val_inner_idx = train_test_split(
        train_outer_idx, test_size=VAL_SPLIT_SIZE, shuffle=True, stratify=train_outer_labels, random_state=RANDOM_SEED
    )

    # Create subsets from the DEVELOPMENT dataset
    train_inner_subset = Subset(dev_dataset, train_inner_idx)
    val_inner_subset = Subset(dev_dataset, val_inner_idx)
    test_outer_subset = Subset(dev_dataset, test_outer_idx) # This is the "test set" for this fold

    train_loader = DataLoader(train_inner_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
    val_loader = DataLoader(val_inner_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)
    test_loader = DataLoader(test_outer_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)

    model = Model(IMG_SIZE, IMG_SIZE,beta=0, batch_size=BATCH_SIZE).to(DEVICE)
    model.load_state_dict(torch.load(transfer_model_path))  # Load the pre-trained model

    optimizer = RAdam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.NLLLoss()
    scheduler = None
    scaler = None

    best_model_path = f"./saved_models/{model_folder_name}/best_model_fold_{fold+1}.pth"
    os.makedirs(f"./saved_models/{model_folder_name}", exist_ok=True)
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    print(f"Training on {len(train_inner_subset)}, Validating on {len(val_inner_subset)}...")
    for epoch in range(1, EPOCHS + 1):
        print(f"\nEpoch {epoch}/{EPOCHS} for Fold {fold+1}")
        train_start_time = time.time()
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        train_end_time = time.time()
        train_time = train_end_time - train_start_time

        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Time: {train_time:.2f}s")

        writer.add_scalar('CV/Loss/train', train_loss, epoch)
        writer.add_scalar('CV/Accuracy/train', train_acc, epoch)
        writer.add_scalar('CV/Loss/val', val_loss, epoch)
        writer.add_scalar('CV/Accuracy/val', val_acc, epoch)
        writer.add_scalar('CV/Time/train', train_time, epoch)

        memory_allocated = torch.cuda.memory_allocated(DEVICE) if torch.cuda.is_available() else 0
        memory_reserved = torch.cuda.memory_reserved(DEVICE) if torch.cuda.is_available() else 0

        writer.add_scalar('CV/Memory/allocated', memory_allocated, epoch)
        writer.add_scalar('CV/Memory/reserved', memory_reserved, epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  New best model for fold {fold+1} at epoch {epoch}: Val Acc: {val_acc:.4f}")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    # Evaluate the best model for this fold on the outer test set
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    model.load_state_dict(torch.load(best_model_path))
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance on Outer Test Set for fold {fold+1}: {final_fold_test_acc:.4f}")
    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('CV/Accuracy/test', final_fold_test_acc, epoch)
    writer.close()

mean_cv_acc = np.mean(fold_test_results)
std_cv_acc = np.std(fold_test_results)
print(f"\n--- NESTED CROSS VALIDATION TRANSFER COMPLETE ---")
print(f"Cross-validation accuracies on outer folds: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Mean CV Accuracy: {mean_cv_acc:.4f} ± {std_cv_acc:.4f}")


========================= PERFORMING NESTED CROSS-VALIDATION TRANSFER =========================

--- Outer Fold 1/3 ---
Training on 2640, Validating on 660...

Epoch 1/100 for Fold 1


Training:   0%|          | 0/21 [00:00<?, ?it/s]/vol/bitbucket/icc24/MSRGNN/RADIO/mxgnet.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(h_score)
Training: 100%|██████████| 21/21 [00:07<00:00,  2.77it/s]


Epoch Loss: 2.8464, Accuracy: 0.1299


Validating: 100%|██████████| 6/6 [00:01<00:00,  5.62it/s]


  Train Loss: 2.8464, Train Acc: 0.1299, Val Loss: 2.4943, Val Acc: 0.1515, Time: 7.58s
  New best model for fold 1 at epoch 1: Val Acc: 0.1515

Epoch 2/100 for Fold 1


...

Validating: 100%|██████████| 6/6 [00:01<00:00,  5.65it/s]


  Train Loss: 1.1277, Train Acc: 0.6193, Val Loss: 1.9286, Val Acc: 0.2939, Time: 7.56s
  Early stopping at epoch 21.
Loading best model for fold 3 (achieved 0.3030 on inner val set).


Validating: 100%|██████████| 13/13 [00:01<00:00,  7.39it/s]

Performance on Outer Test Set for fold 3: 0.2509

--- NESTED CROSS VALIDATION TRANSFER COMPLETE ---
Cross-validation accuracies on outer folds: ['0.2745', '0.2770', '0.2509']
Mean CV Accuracy: 0.2675 ± 0.0118


: 